In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

In [ ]:
import pandas as pd
import numpy as np

cols_rec0111 = ['CASEID', 'V024', 'V025', 'V106', 'V101', 'V102', 'V113',
                'V115', 'V136', 'V161', 'V190']
cols_rec42=['CASEID','V453', 'V457','V457A', 'V459', 'V462', 'V466', 'V481']
cols_rec43 = ['CASEID', 'HIDX','H34', 'H33D','H33M','H33Y', 'H35','H41B', 'H42']
# Anemia, agrego  H35: Cualquier vacuna en los últimos 2 años forma parte de la campaña
cols_rec44 = ['CASEID','HWIDX','HW53', 'HW57A','HW70']
cols_rec91 = ['CASEID', 'S704N','S490AA', 'S490AB', 'S490AC','S490AD','S490AE', 'S490AF']
#rec42: madres
#variable target: V457A: Nivel de Anemia NUEVA DIRECTRIZ OMS 2024/RM 251-2024-MINSA
#otra que agregué por si acaso es V457: Nivel de anemia RM 363-2022-MINSA

#rec44: hijos
#HW57A: Nivel de Anemia NUEVA DIRECTRIZ OMS 2024/RM 251-2024-MINSA

df0111 = pd.read_csv('REC0111_2024.csv', na_values=[" ", "", "NA", "NaN", "nan"], usecols=cols_rec0111)
df42 = pd.read_csv('REC42_2024.csv',  na_values=[" ", "", "NA", "NaN", "nan"],usecols=cols_rec42)
df91 = pd.read_csv('REC91_2024.csv',  na_values=[" ", "", "NA", "NaN", "nan"],usecols=cols_rec91)
df43 = pd.read_csv('REC43_2024.csv',  na_values=[" ", "", "NA", "NaN", "nan"],usecols=cols_rec43)
df44 = pd.read_csv('REC44_2024.csv', na_values=[" ", "", "NA", "NaN", "nan"], usecols=cols_rec44)

In [ ]:
print("Duplicados en df0111_f:", df0111['CASEID'].duplicated().sum())
print("Duplicados en df42_f:", df42['CASEID'].duplicated().sum())
print("Duplicados en df91_f:", df91['CASEID'].duplicated().sum())
print("Duplicados en df43:", df43['CASEID'].duplicated().sum())
print("Duplicados en df44:", df44['CASEID'].duplicated().sum())

Duplicados en df0111_f: 0
Duplicados en df42_f: 0
Duplicados en df91_f: 0
Duplicados en df43: 2143
Duplicados en df44: 2143


In [ ]:
df21

,CASEID,B4,B5,BEDAD,QULT5
0,325503101 2,2,1,43.0,1
3,325504701 2,2,1,13.0,1
5,325505001 1,2,1,19.0,1
8,325508901 2,2,1,13.0,1
10,325509701 2,1,1,18.0,1
...,...,...,...,...,...
60678,705704301 1,2,1,51.0,1
60681,705705001 2,1,1,45.0,1
60684,705705001 6,1,1,22.0,1
60688,705706501 2,1,1,29.0,1


In [ ]:
ids_comunes = set(df0111['CASEID']) & set(df42['CASEID']) & set(df43['CASEID']) & \
              set(df44['CASEID']) & set(df91['CASEID'] ) & set(df21['CASEID'])

print(f"Número de registros comunes en TODOS los archivos: {len(ids_comunes)}")

# Filtrar todos los DataFrames a solo los registros que estén en todos
df0111_f = df0111[df0111['CASEID'].isin(ids_comunes)]
df42_f   = df42[df42['CASEID'].isin(ids_comunes)]
df43_f   = df43[df43['CASEID'].isin(ids_comunes)]
df44_f   = df44[df44['CASEID'].isin(ids_comunes)]
df91_f   = df91[df91['CASEID'].isin(ids_comunes)]

Número de registros comunes en TODOS los archivos: 17567


In [ ]:
#Hacemos merge a todos los datos que se encuentran en los 5 datasets en uno solo df_merged
df_merged = df0111_f.merge(df42_f, on='CASEID') \
                    .merge(df43_f, on='CASEID') \
                    .merge(df44_f, on='CASEID') \
                    .merge(df91_f, on='CASEID')

print("Tamaño final del dataset completo (sin nulos):", df_merged.shape)


Tamaño final del dataset completo (sin nulos): (24181, 37)


In [ ]:
df_merged

,CASEID,V024,V025,V101,V102,V106,V113,V115,V136,V161,...,HW53,HW57A,HW70,S490AA,S490AB,S490AC,S490AD,S490AE,S490AF,S704N
0,325503101 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,143.0,4.0,-88.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
1,325504701 2,1,1,1.0,1.0,2.0,71.0,996.0,4.0,2.0,...,128.0,4.0,-167.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0
2,325505001 1,1,1,1.0,1.0,1.0,11.0,996.0,3.0,2.0,...,125.0,4.0,-78.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
3,325508901 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,104.0,2.0,-27.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0
4,325509701 2,1,1,1.0,1.0,3.0,11.0,996.0,4.0,2.0,...,121.0,4.0,-119.0,1.0,1.0,1.0,0.0,1.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24176,705704301 1,11,1,11.0,1.0,2.0,71.0,1.0,4.0,2.0,...,134.0,4.0,-135.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0
24177,705705001 2,11,1,11.0,1.0,2.0,96.0,1.0,10.0,2.0,...,117.0,4.0,-35.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0
24178,705705001 6,11,1,11.0,1.0,3.0,96.0,1.0,10.0,2.0,...,124.0,4.0,-61.0,1.0,0.0,0.0,0.0,1.0,0.0,4.0
24179,705706501 2,11,1,11.0,1.0,2.0,71.0,30.0,4.0,2.0,...,95.0,2.0,-138.0,1.0,0.0,1.0,0.0,1.0,1.0,2.0


In [ ]:
faltantes = df_merged.isnull().sum()
porcentaje_faltantes = (faltantes / len(df_merged)) * 100

resumen = pd.DataFrame({
    "Faltantes": faltantes,
    "Porcentaje": porcentaje_faltantes.round(2)
}).sort_values(by="Porcentaje", ascending=False)

print(resumen)
#no hay:
#H42 Tomando píldoras de hierro, jarabe
#V459 Tener mosquiteros para dormir (informe del hogar)
#V462 Se lavaron las manos antes de preparar la última comida
#H35: Cualquier vacuna en los últimos 2 años forma parte de la campaña
#H33Y: Vitamina A1 año
#H33M: Vitamina A1 mes
#H33D: Vitamina A1 dia
#H34: Vitamina A en los últimos 6 meses
#H41B: Alguna vez recibió la dosis de vitamina A
#B4:Sexo del niño
#B5:el niño está vivo
#BEDAD: meses de nacimiento
#QULT5: el niño es menor de 5 años

        Faltantes  Porcentaje
V462        24181      100.00
H35         24181      100.00
H42         24181      100.00
V459        24181      100.00
H33M        21460       88.75
H33D        21460       88.75
H33Y        21460       88.75
H34         16034       66.31
H41B         4279       17.70
S704N         977        4.04
HW70          731        3.02
HW53          731        3.02
HW57A         731        3.02
V466           74        0.31
S490AE          6        0.02
S490AF          6        0.02
S490AA          6        0.02
S490AC          6        0.02
S490AD          6        0.02
S490AB          6        0.02
CASEID          0        0.00
V102            0        0.00
V101            0        0.00
V025            0        0.00
V024            0        0.00
V481            0        0.00
HIDX            0        0.00
V457A           0        0.00
V457            0        0.00
V113            0        0.00
V115            0        0.00
V106            0        0.00
V136      

In [ ]:
(df_merged['HW57A'].value_counts()/df_merged['HW57A'].value_counts().sum())*100

,count
HW57A,
4.0,61.070362
3.0,21.786780
9.0,9.914712
2.0,7.125800
1.0,0.102345


In [ ]:
(df_merged['HW57A'].value_counts())

,count
HW57A,
4.0,14321
3.0,5109
9.0,2325
2.0,1671
1.0,24


In [ ]:
# Calcular el porcentaje de valores no nulos por columna
columnas_completas = df_merged.columns[df_merged.isnull().mean() <= 0.20]

In [ ]:
columnas_completas

Index(['CASEID', 'V024', 'V025', 'V101', 'V102', 'V106', 'V113', 'V115',
       'V136', 'V161', 'V190', 'V453', 'V457', 'V457A', 'V466', 'V481', 'HIDX',
       'H41B', 'HWIDX', 'HW53', 'HW57A', 'HW70', 'S490AA', 'S490AB', 'S490AC',
       'S490AD', 'S490AE', 'S490AF', 'S704N'],
      dtype='object')

In [ ]:
(df_merged['V457A'].value_counts()/df_merged['V457A'].value_counts().sum())*100

,count
V457A,
4,70.791117
3,19.143129
2,7.625822
9,2.162855
1,0.277077


In [ ]:
columnas_completas

Index(['CASEID', 'V024', 'V025', 'V101', 'V102', 'V106', 'V113', 'V115',
       'V136', 'V161', 'V190', 'V453', 'V457', 'V457A', 'V466', 'V481', 'HIDX',
       'H41B', 'HWIDX', 'HW53', 'HW57A', 'HW70', 'S490AA', 'S490AB', 'S490AC',
       'S490AD', 'S490AE', 'S490AF', 'S704N'],
      dtype='object')

In [ ]:
df_completas = df_merged[columnas_completas]

In [ ]:
(df_completas['V457A'].value_counts()/df_completas['V457A'].value_counts().sum())

,count
V457A,
4,0.707911
3,0.191431
2,0.076258
9,0.021629
1,0.002771


In [ ]:
(df_completas['V457A'].value_counts())

,count
V457A,
4,17118
3,4629
2,1844
9,523
1,67


In [ ]:
df_completas

,CASEID,V024,V025,V101,V102,V106,V113,V115,V136,V161,...,HW53,HW57A,HW70,S490AA,S490AB,S490AC,S490AD,S490AE,S490AF,S704N
0,325503101 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,143.0,4.0,-88.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
1,325504701 2,1,1,1.0,1.0,2.0,71.0,996.0,4.0,2.0,...,128.0,4.0,-167.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0
2,325505001 1,1,1,1.0,1.0,1.0,11.0,996.0,3.0,2.0,...,125.0,4.0,-78.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
3,325508901 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,104.0,2.0,-27.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0
4,325509701 2,1,1,1.0,1.0,3.0,11.0,996.0,4.0,2.0,...,121.0,4.0,-119.0,1.0,1.0,1.0,0.0,1.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24176,705704301 1,11,1,11.0,1.0,2.0,71.0,1.0,4.0,2.0,...,134.0,4.0,-135.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0
24177,705705001 2,11,1,11.0,1.0,2.0,96.0,1.0,10.0,2.0,...,117.0,4.0,-35.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0
24178,705705001 6,11,1,11.0,1.0,3.0,96.0,1.0,10.0,2.0,...,124.0,4.0,-61.0,1.0,0.0,0.0,0.0,1.0,0.0,4.0
24179,705706501 2,11,1,11.0,1.0,2.0,71.0,30.0,4.0,2.0,...,95.0,2.0,-138.0,1.0,0.0,1.0,0.0,1.0,1.0,2.0


In [ ]:
df_completas.count()

,0
CASEID,24181
V024,24181
V025,24181
V101,24181
V102,24181
V106,24181
V113,24181
V115,24181
V136,24181
V161,24181


In [ ]:
#Evitamos repetidos en los hijos
df_limpio = df_completas[df_completas['HIDX'] == df_completas['HWIDX']].copy()

In [ ]:
df_limpio

,CASEID,V024,V025,V101,V102,V106,V113,V115,V136,V161,...,HW53,HW57A,HW70,S490AA,S490AB,S490AC,S490AD,S490AE,S490AF,S704N
0,325503101 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,143.0,4.0,-88.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
1,325504701 2,1,1,1.0,1.0,2.0,71.0,996.0,4.0,2.0,...,128.0,4.0,-167.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0
2,325505001 1,1,1,1.0,1.0,1.0,11.0,996.0,3.0,2.0,...,125.0,4.0,-78.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
3,325508901 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,104.0,2.0,-27.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0
4,325509701 2,1,1,1.0,1.0,3.0,11.0,996.0,4.0,2.0,...,121.0,4.0,-119.0,1.0,1.0,1.0,0.0,1.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24176,705704301 1,11,1,11.0,1.0,2.0,71.0,1.0,4.0,2.0,...,134.0,4.0,-135.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0
24177,705705001 2,11,1,11.0,1.0,2.0,96.0,1.0,10.0,2.0,...,117.0,4.0,-35.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0
24178,705705001 6,11,1,11.0,1.0,3.0,96.0,1.0,10.0,2.0,...,124.0,4.0,-61.0,1.0,0.0,0.0,0.0,1.0,0.0,4.0
24179,705706501 2,11,1,11.0,1.0,2.0,71.0,30.0,4.0,2.0,...,95.0,2.0,-138.0,1.0,0.0,1.0,0.0,1.0,1.0,2.0


In [ ]:
df_limpio = df_limpio.dropna()

In [ ]:
df_limpio

,CASEID,V024,V025,V101,V102,V106,V113,V115,V136,V161,...,HW53,HW57A,HW70,S490AA,S490AB,S490AC,S490AD,S490AE,S490AF,S704N
0,325503101 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,143.0,4.0,-88.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
1,325504701 2,1,1,1.0,1.0,2.0,71.0,996.0,4.0,2.0,...,128.0,4.0,-167.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0
2,325505001 1,1,1,1.0,1.0,1.0,11.0,996.0,3.0,2.0,...,125.0,4.0,-78.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
3,325508901 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,104.0,2.0,-27.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0
4,325509701 2,1,1,1.0,1.0,3.0,11.0,996.0,4.0,2.0,...,121.0,4.0,-119.0,1.0,1.0,1.0,0.0,1.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24176,705704301 1,11,1,11.0,1.0,2.0,71.0,1.0,4.0,2.0,...,134.0,4.0,-135.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0
24177,705705001 2,11,1,11.0,1.0,2.0,96.0,1.0,10.0,2.0,...,117.0,4.0,-35.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0
24178,705705001 6,11,1,11.0,1.0,3.0,96.0,1.0,10.0,2.0,...,124.0,4.0,-61.0,1.0,0.0,0.0,0.0,1.0,0.0,4.0
24179,705706501 2,11,1,11.0,1.0,2.0,71.0,30.0,4.0,2.0,...,95.0,2.0,-138.0,1.0,0.0,1.0,0.0,1.0,1.0,2.0


In [ ]:
df_limpio.size

441206

In [ ]:
df_limpio.drop('HIDX', axis=1, inplace=True)

/tmp/ipython-input-4144911204.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio.drop('HIDX', axis=1, inplace=True)


In [ ]:
df_limpio.describe().T

,count,mean,std,min,25%,50%,75%,max
V024,15214.0,13.930787,6.856216,1.0,9.0,15.0,20.0,25.0
V025,15214.0,1.271592,0.444795,1.0,1.0,1.0,2.0,2.0
V101,15214.0,13.930787,6.856216,1.0,9.0,15.0,20.0,25.0
V102,15214.0,1.271592,0.444795,1.0,1.0,1.0,2.0,2.0
V106,15214.0,2.216117,0.702299,0.0,2.0,2.0,3.0,3.0
V113,15214.0,28.583016,27.744542,11.0,11.0,11.0,61.0,97.0
V115,15214.0,803.017352,391.518644,1.0,996.0,996.0,996.0,997.0
V136,15214.0,4.864796,1.780296,2.0,4.0,4.0,6.0,19.0
V161,15214.0,4.584067,11.024011,1.0,2.0,2.0,3.0,97.0
V190,15214.0,2.536874,1.305722,1.0,1.0,2.0,4.0,5.0


In [ ]:
(df_limpio['HW57A'].value_counts()/df_limpio['HW57A'].value_counts().sum())*100

,count
HW57A,
4.0,61.502563
3.0,20.855791
9.0,10.858420
2.0,6.684633
1.0,0.098593


In [ ]:
df_limpio['HW57A'].value_counts()

,count
HW57A,
4.0,9357
3.0,3173
9.0,1652
2.0,1017
1.0,15


In [ ]:
df_limpio['V457A'].value_counts()

,count
V457A,
4,11086
3,2754
2,1140
9,186
1,48


In [ ]:
# Columnas binarias a limpiar
binarias = ['S490AC','S490AE','S490AB','S490AF','S490AD','S490AA']

In [ ]:
df_limpio[binarias].sum()

,0
S490AC,11005.0
S490AE,11457.0
S490AB,4284.0
S490AF,5025.0
S490AD,4708.0
S490AA,11750.0


In [ ]:
df_limpio['indice_lavado_manos'] = df_limpio[binarias].sum(axis=1)

/tmp/ipython-input-2064859319.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['indice_lavado_manos'] = df_limpio[binarias].sum(axis=1)


In [ ]:
df_limpio.drop(df_limpio[binarias], axis=1, inplace=True)

/tmp/ipython-input-380076668.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio.drop(df_limpio[binarias], axis=1, inplace=True)


In [ ]:
#Ahora se va a promediar los valores
#HW1
#HW2
#HW3
#HW53: Nivel de hemoglobina (g /dl -1 decimal)
#HW57A:Nivel de Anemia NUEVA DIRECTRIZ OMS 2024/RM 251-2024-MINSA
cols_antropo  = ['HW53']
col_anemia  = ['HW57A']
cols_constantes = [
    'CASEID', 'V024', 'V025', 'V101', 'V102', 'V106', 'V113', 'V115',
    'V136', 'V161', 'V190', 'V453', 'V457', 'V457A', 'V466', 'V481',
     'indice_lavado_manos', 'S704N'
]

In [ ]:
# Promedios por CASEID
df_promedios = df_limpio.groupby('CASEID')[cols_antropo].mean().reset_index()

In [ ]:
df_promedios

,CASEID,HW53
0,325503101 2,143.0
1,325504701 2,128.0
2,325505001 1,125.0
3,325508901 2,104.0
4,325509701 2,121.0
...,...,...
13761,705704301 1,134.0
13762,705705001 2,117.0
13763,705705001 6,124.0
13764,705706501 2,95.0


In [ ]:
df_limpio

,CASEID,V024,V025,V101,V102,V106,V113,V115,V136,V161,...,V457A,V466,V481,H41B,HWIDX,HW53,HW57A,HW70,S704N,indice_lavado_manos
0,325503101 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,4,1.0,1,0.0,1,143.0,4.0,-88.0,1.0,3.0
1,325504701 2,1,1,1.0,1.0,2.0,71.0,996.0,4.0,2.0,...,4,1.0,1,0.0,1,128.0,4.0,-167.0,2.0,2.0
2,325505001 1,1,1,1.0,1.0,1.0,11.0,996.0,3.0,2.0,...,4,1.0,1,0.0,1,125.0,4.0,-78.0,1.0,2.0
3,325508901 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,3,1.0,1,0.0,1,104.0,2.0,-27.0,2.0,2.0
4,325509701 2,1,1,1.0,1.0,3.0,11.0,996.0,4.0,2.0,...,4,1.0,1,0.0,1,121.0,4.0,-119.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24176,705704301 1,11,1,11.0,1.0,2.0,71.0,1.0,4.0,2.0,...,4,1.0,1,1.0,1,134.0,4.0,-135.0,2.0,5.0
24177,705705001 2,11,1,11.0,1.0,2.0,96.0,1.0,10.0,2.0,...,3,1.0,1,1.0,1,117.0,4.0,-35.0,2.0,2.0
24178,705705001 6,11,1,11.0,1.0,3.0,96.0,1.0,10.0,2.0,...,4,1.0,1,1.0,1,124.0,4.0,-61.0,4.0,2.0
24179,705706501 2,11,1,11.0,1.0,2.0,71.0,30.0,4.0,2.0,...,4,1.0,1,0.0,1,95.0,2.0,-138.0,2.0,4.0


In [ ]:
df_anemia = df_limpio.groupby('CASEID')[col_anemia].max().reset_index()

In [ ]:
df_anemia

,CASEID,HW57A
0,325503101 2,4.0
1,325504701 2,4.0
2,325505001 1,4.0
3,325508901 2,2.0
4,325509701 2,4.0
...,...,...
13761,705704301 1,4.0
13762,705705001 2,4.0
13763,705705001 6,4.0
13764,705706501 2,2.0


In [ ]:
(df_anemia['HW57A'].value_counts()/df_anemia['HW57A'].value_counts().sum())

,count
HW57A,
4.0,0.626398
3.0,0.197370
9.0,0.117536
2.0,0.057969
1.0,0.000726


In [ ]:
df_anemia['HW57A'].value_counts()

,count
HW57A,
4.0,8623
3.0,2717
9.0,1618
2.0,798
1.0,10


In [ ]:
df_limpio.T

,0,1,2,3,4,5,8,11,15,22,...,24166,24169,24170,24171,24173,24176,24177,24178,24179,24180
CASEID,325503101 2,325504701 2,325505001 1,325508901 2,325509701 2,325600801 2,325600801 2,325603201 3,325800601 4,325809301 1,...,651009901 2,651108801 2,651110001 5,651111001 1,651112001 2,705704301 1,705705001 2,705705001 6,705706501 2,705708801 1
V024,1,1,1,1,1,1,1,1,1,1,...,25,13,13,13,13,11,11,11,11,11
V025,1,1,1,1,1,1,1,1,1,1,...,2,1,1,1,1,1,1,1,1,1
V101,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,25.0,13.0,13.0,13.0,13.0,11.0,11.0,11.0,11.0,11.0
V102,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
V106,2.0,2.0,1.0,2.0,3.0,3.0,3.0,3.0,1.0,2.0,...,1.0,2.0,3.0,2.0,1.0,2.0,2.0,3.0,2.0,3.0
V113,11.0,71.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,...,43.0,11.0,11.0,12.0,11.0,71.0,96.0,96.0,71.0,11.0
V115,996.0,996.0,996.0,996.0,996.0,996.0,996.0,996.0,996.0,996.0,...,20.0,996.0,996.0,30.0,996.0,1.0,1.0,1.0,30.0,996.0
V136,4.0,4.0,3.0,4.0,4.0,4.0,4.0,7.0,5.0,7.0,...,9.0,3.0,7.0,3.0,4.0,4.0,10.0,10.0,4.0,4.0
V161,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,...,8.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0


In [ ]:
print("Duplicados en df43_f:", (df_limpio[cols_constantes]).duplicated().sum())

Duplicados en df43_f: 1448


In [ ]:
df_limpio.to_csv("datos_limpios.csv", index=False)

In [ ]:
df_constantes = df_limpio[cols_constantes].drop_duplicates(subset='CASEID')

In [ ]:
print("Duplicados en df43_f:", (df_constantes['CASEID']).duplicated().sum())

Duplicados en df43_f: 0


In [ ]:
df_constantes

,CASEID,V024,V025,V101,V102,V106,V113,V115,V136,V161,V190,V453,V457,V457A,V466,V481,indice_lavado_manos,S704N
0,325503101 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,2,147,4,4,1.0,1,3.0,1.0
1,325504701 2,1,1,1.0,1.0,2.0,71.0,996.0,4.0,2.0,2,143,4,4,1.0,1,2.0,2.0
2,325505001 1,1,1,1.0,1.0,1.0,11.0,996.0,3.0,2.0,2,139,4,4,1.0,1,2.0,1.0
3,325508901 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,3,129,3,3,1.0,1,2.0,2.0
4,325509701 2,1,1,1.0,1.0,3.0,11.0,996.0,4.0,2.0,4,144,4,4,1.0,1,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24176,705704301 1,11,1,11.0,1.0,2.0,71.0,1.0,4.0,2.0,2,133,4,4,1.0,1,5.0,2.0
24177,705705001 2,11,1,11.0,1.0,2.0,96.0,1.0,10.0,2.0,2,111,3,3,1.0,1,2.0,2.0
24178,705705001 6,11,1,11.0,1.0,3.0,96.0,1.0,10.0,2.0,2,126,4,4,1.0,1,2.0,4.0
24179,705706501 2,11,1,11.0,1.0,2.0,71.0,30.0,4.0,2.0,2,133,4,4,1.0,1,4.0,2.0


In [ ]:
df_final = df_constantes.merge(df_promedios, on='CASEID').merge(df_anemia, on='CASEID')

In [ ]:
df_final.to_csv("anemia_datos_completos.csv", index=False)

PREPROCESAMIENTO

In [ ]:
df_final.T

,0,1,2,3,4,5,6,7,8,9,...,13756,13757,13758,13759,13760,13761,13762,13763,13764,13765
CASEID,325503101 2,325504701 2,325505001 1,325508901 2,325509701 2,325600801 2,325603201 3,325800601 4,325809301 1,325903201 2,...,651009901 2,651108801 2,651110001 5,651111001 1,651112001 2,705704301 1,705705001 2,705705001 6,705706501 2,705708801 1
V024,1,1,1,1,1,1,1,1,1,1,...,25,13,13,13,13,11,11,11,11,11
V025,1,1,1,1,1,1,1,1,1,1,...,2,1,1,1,1,1,1,1,1,1
V101,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,25.0,13.0,13.0,13.0,13.0,11.0,11.0,11.0,11.0,11.0
V102,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
V106,2.0,2.0,1.0,2.0,3.0,3.0,3.0,1.0,2.0,3.0,...,1.0,2.0,3.0,2.0,1.0,2.0,2.0,3.0,2.0,3.0
V113,11.0,71.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,71.0,...,43.0,11.0,11.0,12.0,11.0,71.0,96.0,96.0,71.0,11.0
V115,996.0,996.0,996.0,996.0,996.0,996.0,996.0,996.0,996.0,996.0,...,20.0,996.0,996.0,30.0,996.0,1.0,1.0,1.0,30.0,996.0
V136,4.0,4.0,3.0,4.0,4.0,4.0,7.0,5.0,7.0,4.0,...,9.0,3.0,7.0,3.0,4.0,4.0,10.0,10.0,4.0,4.0
V161,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,...,8.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0


In [ ]:
df_final.describe().T #para la transpuesta

,count,mean,std,min,25%,50%,75%,max
V024,13766.0,13.871422,6.852864,1.0,9.0,15.0,19.0,25.0
V025,13766.0,1.266381,0.442082,1.0,1.0,1.0,2.0,2.0
V101,13766.0,13.871422,6.852864,1.0,9.0,15.0,19.0,25.0
V102,13766.0,1.266381,0.442082,1.0,1.0,1.0,2.0,2.0
V106,13766.0,2.227372,0.699142,0.0,2.0,2.0,3.0,3.0
V113,13766.0,28.474866,27.724529,11.0,11.0,11.0,61.0,97.0
V115,13766.0,806.980677,388.496119,1.0,996.0,996.0,996.0,997.0
V136,13766.0,4.789191,1.753793,2.0,4.0,4.0,6.0,19.0
V161,13766.0,4.552811,11.039404,1.0,2.0,2.0,3.0,97.0
V190,13766.0,2.561238,1.305104,1.0,1.0,2.0,4.0,5.0


In [ ]:
(df_limpio['HW57A'].value_counts()/df_limpio['HW57A'].count())*100

,count
HW57A,
4.0,61.502563
3.0,20.855791
9.0,10.858420
2.0,6.684633
1.0,0.098593


In [ ]:
df_limpio.shape

(15214, 23)

In [ ]:
print("Duplicados en df21:", df_final['CASEID'].duplicated().sum())

Duplicados en df21: 0


In [ ]:
df_limpio

,CASEID,V024,V025,V101,V102,V106,V113,V115,V136,V161,...,V457A,V466,V481,H41B,HWIDX,HW53,HW57A,HW70,S704N,indice_lavado_manos
0,325503101 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,4,1.0,1,0.0,1,143.0,4.0,-88.0,1.0,3.0
1,325504701 2,1,1,1.0,1.0,2.0,71.0,996.0,4.0,2.0,...,4,1.0,1,0.0,1,128.0,4.0,-167.0,2.0,2.0
2,325505001 1,1,1,1.0,1.0,1.0,11.0,996.0,3.0,2.0,...,4,1.0,1,0.0,1,125.0,4.0,-78.0,1.0,2.0
3,325508901 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,3,1.0,1,0.0,1,104.0,2.0,-27.0,2.0,2.0
4,325509701 2,1,1,1.0,1.0,3.0,11.0,996.0,4.0,2.0,...,4,1.0,1,0.0,1,121.0,4.0,-119.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24176,705704301 1,11,1,11.0,1.0,2.0,71.0,1.0,4.0,2.0,...,4,1.0,1,1.0,1,134.0,4.0,-135.0,2.0,5.0
24177,705705001 2,11,1,11.0,1.0,2.0,96.0,1.0,10.0,2.0,...,3,1.0,1,1.0,1,117.0,4.0,-35.0,2.0,2.0
24178,705705001 6,11,1,11.0,1.0,3.0,96.0,1.0,10.0,2.0,...,4,1.0,1,1.0,1,124.0,4.0,-61.0,4.0,2.0
24179,705706501 2,11,1,11.0,1.0,2.0,71.0,30.0,4.0,2.0,...,4,1.0,1,0.0,1,95.0,2.0,-138.0,2.0,4.0
